In [2]:
import json
import os
import pandas as pd


with open('AI-Judger.json', 'r') as f:
    golds = json.load(f)
    
# flaw_q = [14, 15, 17, 44, 47, 48, 49, 50, 58]
# if gold['Q#'] not in flaw_q

gold_id2ans = {gold['Q#']: gold['agreement']['answer'] for gold in golds }

root = "results/60-qs"
# responses_paths = [f"{root}/{file}" for file in os.listdir(root) if ".csv" in file]

dfs = {file[:-4]: pd.read_csv(f"{root}/{file}") for file in os.listdir(root) if ".csv" in file}

In [3]:
dfs.keys()

dict_keys(['Qwen2.5-0.5B-Instruct', 'Llama-3.3-70B-Instruct', 'Llama-3.1-8B-Instruct', 'Qwen2.5-72B-Instruct', 'Qwen2.5-7B-Instruct', 'Llama-3.2-1B-Instruct'])

In [4]:
llm = 'Qwen2.5-0.5B-Instruct'

dfs[llm].head(3)

,id,question,category,standard-answer,standard-eval,domain-answer,domain-eval,rdf-onto-answer,rdf-onto-eval,omn-onto-answer,omn-onto-eval,ofn-onto-answer,ofn-onto-eval,ttl-onto-answer,ttl-onto-eval
0,1,What are the common toppings of a Margherita P...,IND,A common topping of a Margherita pizza is toma...,1,The common toppings of a Margherita Pizza incl...,1,Provide a description from a Margherita Pizza\...,1,"# owl 的 1,3\n # owl, 3\n ...",1,Tiffany's pizza is a popular choice due to its...,1,"The following are examples of : SpicyPizza, : ...",1
1,2,Is a FourCheesePizza considered a VegetarianPi...,SUB,"No, a FourCheesePizza is not considered a Vege...",1,"No, a FourCheesePizza is not considered a vege...",1,Is a FourCheesePizza considered a VegetarianPi...,1,"T-shirt, a pair of scissors&t-shirt\n\n# 咖啡豆, ...",1,"No, it is not a vegetarian or vegan pizza. It ...",1,"OK,`:` ; ` `; ` ; ` ; ` ; ` ; ` ; ` ; ` ; ` ; ...",1
2,3,What toppings make a pizza classified as a Hot...,SUB,A hot and spicy pizza typically has several to...,1,A hot and spicy pizza is typically made with:\...,1,Provide a pizza classified as a HighAndSpicyPi...,1,"<Twitter问题的表达方式为：""在中文，we recommend you read th...",1,Any food or drink that does not contain any se...,1,":VegetarianTopping, :MozhTopping, :Tomahping, ...",1


In [22]:
cols = {
    "standard-answer": "standard-eval",
    "domain-answer": "domain-eval",
    "rdf-onto-answer": "rdf-onto-eval",
    "omn-onto-answer": "omn-onto-eval",
    "ofn-onto-answer": "ofn-onto-eval",
    "ttl-onto-answer": "ttl-onto-eval"
}

In [24]:
from openai import OpenAI
import config
import numpy as np

client = OpenAI(api_key=config.openai_token)

def get_embedding(text, model='text-embedding-3-large'):
    text = str(text)
    return client.embeddings.create(input = [text], model=model).data[0].embedding


In [8]:
gols_embeddings = {gold_id: get_embedding(text=gold_ans) for gold_id, gold_ans in gold_id2ans.items()}

In [25]:
for response_col, eval_col in cols.items():
    print("working on:", response_col)
    for llm, df in dfs.items():
        print("\t working on:", llm)
        embeddings = df[response_col].apply(lambda x: get_embedding(x, model='text-embedding-3-large'))
        embeddings_np = np.array([embed for embed in embeddings])
        gols_embeddings_np = np.array(list(gols_embeddings.values()))
        A_norm = gols_embeddings_np / np.linalg.norm(gols_embeddings_np, axis=1, keepdims=True)
        B_norm = embeddings_np / np.linalg.norm(embeddings_np, axis=1, keepdims=True)
        cosine_similarities = np.sum(A_norm * B_norm, axis=1)
        dfs[llm][eval_col] = list(cosine_similarities)

working on: omn-onto-answer
	 working on: Qwen2.5-0.5B-Instruct
	 working on: Llama-3.3-70B-Instruct
	 working on: Llama-3.1-8B-Instruct
	 working on: Qwen2.5-72B-Instruct
	 working on: Qwen2.5-7B-Instruct
	 working on: Llama-3.2-1B-Instruct
working on: ofn-onto-answer
	 working on: Qwen2.5-0.5B-Instruct
	 working on: Llama-3.3-70B-Instruct
	 working on: Llama-3.1-8B-Instruct
	 working on: Qwen2.5-72B-Instruct
	 working on: Qwen2.5-7B-Instruct
	 working on: Llama-3.2-1B-Instruct
working on: ttl-onto-answer
	 working on: Qwen2.5-0.5B-Instruct
	 working on: Llama-3.3-70B-Instruct
	 working on: Llama-3.1-8B-Instruct
	 working on: Qwen2.5-72B-Instruct
	 working on: Qwen2.5-7B-Instruct
	 working on: Llama-3.2-1B-Instruct


In [26]:
for llm, df in dfs.items():
    df.to_csv(f"results/{llm}-evaluated.csv", index=False)